In [4]:
import pandas as pd 
import numpy as np

In [5]:
'''
!pip install requests 
!pip install beautifulsoup4 
!pip install lxml
'''

In [6]:
from bs4 import BeautifulSoup
import requests 

In [7]:
root = 'https://www.boxofficemojo.com/weekly/by-year'
s = requests.get(root).text
sou = BeautifulSoup(s, 'lxml')
t = sou.find('table')

headers = []
for h in t.find_all('th'):
    x = h.text
    headers.append(x)

a = []
for i in range(47):
    a.append(1977 + i)

links = []
for i in a:
    date = str(i)
    links.append(date + '/')

d = {}

In [8]:
for link in links:
    source = requests.get(f'{root}/{link}')
    content = source.text
    soup = BeautifulSoup(content, 'lxml')
    table = soup.find('table')
    d[link] = pd.DataFrame(columns = headers)
    for rows in table.find_all('tr')[1:]:
        row_values = rows.find_all('td')
        values = [i.text for i in row_values]
        entry = len(d[link])
        d[link].loc[entry] = values

In [13]:
df_headers = ['Week', 'Name', 'Week_Gross', 'Category']
final_df = pd.DataFrame(columns = df_headers)
for link in links:
    aux = pd.DataFrame(columns = df_headers)
    aux['Week'] = d[link]['Week']
    aux['Name'] = d[link]['#1 Release\n']
    aux['Week_Gross'] = d[link]['Overall Gross']
    aux['Category'] = link
    final_df = pd.concat([final_df, aux], ignore_index = True)

In [15]:
final_df['Week'] = final_df['Week'].astype(int)
final_df = final_df.sort_values(by= 'Week')
final_df.reset_index(inplace = True)
final_df.drop(columns = 'index', inplace = True)

In [16]:
final_df['Week_Gross'] = final_df['Week_Gross'].astype(str)
final_df['Week_Gross'] = final_df['Week_Gross'].str.replace('$', '')
final_df['Week_Gross'] = final_df['Week_Gross'].str.replace(',', '')
final_df['Week_Gross'] = final_df['Week_Gross'].str.replace('-', '0')

In [18]:
caption_list = (final_df.groupby('Week').size()).copy()
caption_list = caption_list.apply(str)
new_index = caption_list.index.astype('str')
caption_list = [c for c in caption_list]
new_index = ['Week ' + i for i in new_index]
caption_list = pd.Series(caption_list, index = new_index)

In [32]:
# caption_list 

Week 1     44
Week 2     43
Week 3     45
Week 4     44
Week 5     46
Week 6     46
Week 7     46
Week 8     45
Week 9     47
Week 10    47
Week 11    47
Week 12    45
Week 13    46
Week 14    45
Week 15    44
Week 16    46
Week 17    45
Week 18    45
Week 19    46
Week 20    46
Week 21    46
Week 22    46
Week 23    47
Week 24    47
Week 25    47
Week 26    47
Week 27    47
Week 28    46
Week 29    47
Week 30    47
Week 31    46
Week 32    46
Week 33    46
Week 34    46
Week 35    45
Week 36    44
Week 37    47
Week 38    45
Week 39    44
Week 40    46
Week 41    44
Week 42    44
Week 43    45
Week 44    47
Week 45    45
Week 46    46
Week 47    45
Week 48    43
Week 49    46
Week 50    46
Week 51    46
Week 52    46
Week 53     8
dtype: object

In [20]:
for column in final_df.columns:
    final_df[column] = final_df[column].apply(str)

In [21]:
final_df['Week'] = 'Week ' + final_df['Week']

In [27]:
final_df1 = pd.DataFrame(columns = df_headers)
n = len(final_df)
empty = {'Week'       : '',
         'Name'       : '',
         'Week_Gross' : '',
         'Category'   : ''
        }
for i in range(n-1):
    entry = len(final_df1)
    if (final_df.loc[i]['Week'] == final_df.loc[i + 1]['Week']):
        final_df1.loc[entry] = final_df.loc[i]
    else:
        r = final_df.loc[i + 1]['Week']
        s = caption_list.loc[r]
        caption = {'Week'       : s,
                   'Name'       : '',
                   'Week_Gross' : '',
                   'Category'   : ''
                  }
        final_df1.loc[entry] = final_df.loc[i]
        final_df1.loc[entry + 1] = empty 
        final_df1.loc[entry + 2] = caption 

In [33]:
final_df1.loc[len(final_df1)] = final_df.loc[len(final_df) - 1]

In [34]:
pwd

'/kaggle/working'

In [41]:
file = open('/kaggle/working/Largest_Week_Gross.txt', 'w+')
final_df1.to_csv(file, sep = ',', index = False)

In [42]:
with open('/kaggle/working/Largest_Week_Gross.txt', 'r') as file:
    text = file.read()
    text = text.replace(',,,', '')

with open('/kaggle/working/Largest_Week_Gross.txt', 'w+') as file:
    file.write(text)
    file.close()